In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext,SparkSession
from pyspark.sql.types import *
from sagemaker_pyspark import IAMRole, classpath_jars
from sagemaker_pyspark.algorithms import KMeansSageMakerEstimator

In [2]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/ec2-user/snowflake/spark-snowflake_2.11-2.8.1-spark_2.4.jar,/home/ec2-user/snowflake/snowflake-jdbc-3.12.9.jar pyspark-shell'

In [3]:
conf = (SparkConf()
        #.set("spark.driver.extraClassPath", (":".join(classpath_jars())+":"+":".join(sfc_jars)))
        .setMaster('local')
        .setAppName('local-spark-test'))
sc=SparkContext.getOrCreate(conf=conf)
sc.stop()

sc=SparkContext(conf=conf)
print (sc)

spark = SQLContext(sc)
print (spark)

<SparkContext master=local appName=local-spark-test>


In [4]:
sfOptions = {
  "sfURL" : "https://oq51261.eu-west-1.snowflakecomputing.com/",
  "sfAccount" : "oq51261",
  "sfUser" : "sagemaker",
  "sfPassword" : "IRISPW",
  "sfDatabase" : "SNOWFLAKE_SAMPLE_DATA",
  "sfSchema" : "WEATHER",
  "sfWarehouse" : "SAGEMAKER_WH",
}

In [5]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"
df = spark.read.format(SNOWFLAKE_SOURCE_NAME) \
  .options(**sfOptions) \
  .option("query", \
"select (V:main.temp_max - 273.15) * 1.8000 + 32.00 as temp_max_far, " +\
"       (V:main.temp_min - 273.15) * 1.8000 + 32.00 as temp_min_far, " +\
"       cast(V:time as timestamp) time, " +\
"       V:city.coord.lat lat, " +\
"       V:city.coord.lon lon " +\
"from snowflake_sample_data.weather.weather_14_total limit 50000000").load()

In [6]:
df.count()

50000000

In [10]:
import time

t0 = time.time()
df.write.csv('weather_50mn_records.csv')
t1 = time.time()

Py4JJavaError: An error occurred while calling o128.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:226)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:154)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:104)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:102)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:122)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.sca

In [ ]:
print ('took %d seconds to save the file'.format(t1-t0))

# Uplaod data to S3

In [ ]:
import sagemaker

session = sagemaker.session.Session()
s3_bucket = 'snowflake-getting-started'
prefix = 'weather/data'

print ('uploading data to s3')
s3_data_path = session.upload_data(path='weather_50mn.csv', 
                                   bucket=s3_bucket, 
                                   key_prefix=prefix)
print ('data uploaded to -', s3_data_path)